Mosfet parser script

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore


service_account_path = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/atopile-880ca67acfe2.json"

if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Reference to the 'mosfets' collection
mosfets_collection = db.collection('mosfets')

# Fetch all documents in the collection
docs = mosfets_collection.stream()


In [ ]:
from pathlib import Path
import subprocess
import glob

def get_footprint_data(jlcpn):
    dir = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/temp"
    command = [
    "easyeda2kicad",
    "--full",
    f"--lcsc_id={jlcpn}",
    f"--output={dir}/temp",
    "--overwrite",
    "--ato",
    f"--ato_file_path={dir}",
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    # read the ato file data glob matching with .ato extension
    ato_file_path = glob.glob(f"{dir}/*.ato")[0]

    # read the kicad_mod file data
    kicad_mod_file_path = glob.glob(f"{dir}/temp.pretty/*.kicad_mod")[0]


    # Read the contents of the .ato and .kicad_mod files
    ato_file_content = read_file(ato_file_path)
    kicad_mod_file_content = read_file(kicad_mod_file_path)

    # Parse the ato file and transform the kicad_mod file
    ato_mapping = parse_ato_file(ato_file_content)
    transformed_kicad_mod = transform_kicad_mod(kicad_mod_file_content, ato_mapping)

    return transformed_kicad_mod

def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def parse_ato_file(ato_content):
    signal_mapping = {}
    for line in ato_content.splitlines():
        if '~ pin' in line:
            parts = line.strip().split(' ~ pin ')
            signal = parts[0].split(' ')[-1]
            pin = 'pin ' + parts[1]
            if signal not in signal_mapping:
                signal_mapping[signal] = []
            signal_mapping[signal].append(pin)
    return signal_mapping

def transform_kicad_mod(kicad_mod_content, ato_mapping):
    desired_mapping = {'G': 'G', 'S': 'S', 'D': 'D'}
    reverse_ato_mapping = {pin: signal for signal, pins in ato_mapping.items() for pin in pins}

    transformed_lines = []
    for line in kicad_mod_content.splitlines():
        if line.strip().startswith('(pad'):
            parts = line.strip().split(' ')
            original_pin = parts[1]
            signal = reverse_ato_mapping.get('pin ' + original_pin)
            if signal and signal in desired_mapping:
                parts[1] = desired_mapping[signal]  # Apply the transformation
            transformed_line = '\t' + ' '.join(parts)  # Adding tab indent
        else:
            transformed_line = '\t' + line.strip()
        transformed_lines.append(transformed_line)
    return '\n'.join(transformed_lines)


# print(get_footprint_data("C131006"))

In [52]:
# Fetch all documents in the collection
docs = mosfets_collection.stream()
dir = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/temp"


import os
import shutil

def delete_contents_in_folder(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return

    # Loop through all the files and directories in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file or directory
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
count = 0

items = {}

for doc in reversed(list(docs)):
    doc_data = doc.to_dict()
    # Your code to process doc_data

    try:
    # Move the existing footprint data to 'footprint_data.kicad'
        footprint_data = get_footprint_data(doc_data['LCSC Part #'])
        # check how many ')' there are in the footprint data

        brackets = footprint_data.count(')')
        if brackets > 10:
            print("I think we already have the footprint data")
            delete_contents_in_folder(dir)

            continue
        doc_data['footprint_data'] = {'kicad': footprint_data}
    except Exception as e:
        print(f"Error updating footprint for {doc_data.get('LCSC Part #', 'Unknown LCSC Part #')}")
        print(e)
        continue

    # Update the document with new data
    mosfets_collection.document(doc.id).set(doc_data, timeout = 10)

    print(f"Updated footprint for {doc_data.get('LCSC Part #', 'Unknown LCSC Part #')}")

    # delete the contents of the temp folder
    #delete everything in the temp folder
    delete_contents_in_folder(dir)

print("Database update completed.")

I think we already have the footprint data
I think we already have the footprint data
I think we already have the footprint data
I think we already have the footprint data
I think we already have the footprint data
I think we already have the footprint data
I think we already have the footprint data
